In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *
from distill.repsim_util import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm

chkpt_dir='../tf_ckpts'

[nltk_data] Downloading package punkt to /home/samira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
task1 = 'lm1b'
task1 = TASKS[task1](get_task_params(batch_size=100), data_dir='../data')
cl_token = task1.sentence_encoder().encode(constants.bos)

In [11]:
for x,y in task1.valid_dataset:
    print(x.shape)
    print(x)
    break

(100, 82)
tf.Tensor(
[[   3 6244   10 ...    1    1    1]
 [   3 7872 7969 ...    1    1    1]
 [   3 2247 3429 ...    1    1    1]
 ...
 [   3 1005 5733 ...    1    1    1]
 [   3 1118   41 ...    1    1    1]
 [   3   19 5311 ...    1    1    1]], shape=(100, 82), dtype=int64)


In [12]:
# LM LSTM
config = {'model_name':'lm_lstm_shared_emb',
        'model_config':'small_lstm_v4',
        'learning_rate':0.001,
        'exp_name':'test',
        'chkpt_dir': '../tf_ckpts'
}
hparams=get_model_params(task1, config['model_name'], config['model_config'])
hparams.output_attentions = True
hparams.output_embeddings = True

lm_lstm_1, ckpt = get_model(config, task1, hparams, cl_token)

model config: small_lstm_v4
{'hidden_dim': 256, 'embedding_dim': 256, 'depth': 2, 'hidden_dropout_rate': 0.8, 'input_dropout_rate': 0.2, 'initializer_range': 0.1}
No checkpoint found ../tf_ckpts/lm1b/lm_lstm_shared_emb_em-256_h-256_d-2_hdrop-0.8_indrop-0.2_small_lstm_v4_0.001_test


In [13]:
task1.input_padding_symbol

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [14]:
task1.output_padding_symbol

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [15]:
loss_fn = task1.get_loss_fn()

In [16]:
logits = lm_lstm_1(x)

In [17]:
loss_fn(x,logits)

<tf.Tensor: shape=(), dtype=float32, numpy=9.095496>

In [19]:
task1.n_train_batches

303010

In [20]:
task1.n_valid_batches

3066

In [21]:
task1.n_test_batches

3066